# Validation of DP and EMT 4th Order VBR SynGen with Controllers against SP

## Import Libraries

In [ ]:
import os
import numpy as np
from villas.dataprocessing.readtools import *
from villas.dataprocessing.timeseries import *
from villas.dataprocessing.timeseries import TimeSeries as ts
import matplotlib.pyplot as plt
import dpsimpy

### Parametrization

In [ ]:
### Power System
frequency = 60
omega = 2 * np.pi * frequency
nominal_voltage_mv = 24e3

### Generator
nom_power = 555e6
initial_electrical_power = complex(300e6, 0)
initial_terminal_voltage = nominal_voltage_mv * complex(np.cos(-np.pi / 2), np.sin(-np.pi / 2))
H = 3.7
Td0_t = 8.0669
Td0_s = 0.0300
Td_t = 1.3368
Td_s = 0.0230
Ld_t = 0.2999
Ld_s = 0.2299
Tq0_t = 0.9991
Tq0_s = 0.0700
Lq_t = 0.6500
Lq_s = 0.2500
Ld = 1.8099
Lq = 1.7600
L0 = 0.15
Taa = 0

### Excitation system
exciter_parameters = dpsimpy.signal.ExciterDC1SimpParameters()
exciter_parameters.Ka = 46
exciter_parameters.Ta = 0.06
exciter_parameters.Kef = -0.0435
exciter_parameters.Tef = 0.46
exciter_parameters.Kf = 0.1
exciter_parameters.Tf = 1
exciter_parameters.Tr = 0.02
exciter_parameters.Aef = 0.33
exciter_parameters.Bef = 0.1
exciter_parameters.MaxVa = 1.0
exciter_parameters.MinVa = -0.9
#exciter_parameters.Tb = 10
#exciter_parameters.Tc = 1.0

### PSS
pss_parameters = dpsimpy.signal.PSS1AParameters()
pss_parameters.Kp = 0
pss_parameters.Kv = 0
pss_parameters.Kw = 15
pss_parameters.T1 = 0.1
pss_parameters.T2 = 0.01
pss_parameters.T3 = 0.12
pss_parameters.T4 = 0.01
pss_parameters.Vs_max = 0.1
pss_parameters.Vs_min = -0.1
pss_parameters.Tw = 10

### Turbine Governor Type 1
governor_parameters = dpsimpy.signal.TurbineGovernorType1Parameters()
governor_parameters.OmRef = 1.0
governor_parameters.R = 0.02
governor_parameters.Pmax = 1.2
governor_parameters.Pmin = 0.3
governor_parameters.Ts = 0.1
governor_parameters.Tc = 0.45
governor_parameters.T3 = 0.0
governor_parameters.T4 = 12.0
governor_parameters.T5 = 50.0

# Load
active_power = 300e6
reactive_power = 0

### Switch
switch_closed = 10
switch_open = 1e6

## Declare DPSim Topology for time domain simulations

In [ ]:
def reducedOrderSG_LoadStep(domain, gen_model, with_exciter=False, with_pss=False, with_governor=False, final_time=10, time_step=0.001):

    ### DPsim SP simulation
    name = ""
    if domain == dpsimpy.Domain.SP:
        name = "SP_ReducedOrderSG_SMIB_" + gen_model
    elif domain == dpsimpy.Domain.DP:
        name = "DP_ReducedOrderSG_SMIB_" + gen_model
    elif domain == dpsimpy.Domain.EMT:
        name = "EMT_ReducedOrderSG_SMIB_" + gen_model
    
    dpsimpy.Logger.set_log_dir("logs/" + name)

    ### Nodes
    n1 = None
    if (domain == dpsimpy.Domain.SP or domain == dpsimpy.Domain.DP):
        n1 = dpsimpy.dp.SimNode('n1', dpsimpy.PhaseType.Single)
        n1.set_initial_voltage(initial_terminal_voltage)
    elif domain == dpsimpy.Domain.EMT:
        n1 = dpsimpy.emt.SimNode('n1', dpsimpy.PhaseType.ABC) 
        n1.set_initial_voltage(dpsimpy.Math.single_phase_variable_to_three_phase(initial_terminal_voltage))
    

    ### Components
    comps_module = None
    if domain == dpsimpy.Domain.SP:
        comps_module = dpsimpy.sp.ph1
    elif domain == dpsimpy.Domain.DP:
        comps_module = dpsimpy.dp.ph1
    elif domain == dpsimpy.Domain.EMT:
        comps_module = dpsimpy.emt.ph3
        
    # syncrhon generator
    gen = None
    if (gen_model=="3Order"):
        gen = comps_module.SynchronGenerator3OrderVBR('gen', dpsimpy.LogLevel.off)
        gen.set_operational_parameters_per_unit(nom_power=nom_power, nom_voltage=nominal_voltage_mv, nom_frequency=frequency, 
                                                H=H, Ld=Ld, Lq=Lq, L0=L0, Ld_t=Ld_t, Td0_t=Td0_t)
    elif (gen_model=="4Order"):
        gen = comps_module.SynchronGenerator4OrderVBR('gen', dpsimpy.LogLevel.info)
        gen.set_operational_parameters_per_unit(nom_power=nom_power, nom_voltage=nominal_voltage_mv, nom_frequency=frequency, 
                                                H=H, Ld=Ld, Lq=Lq, L0=L0, Ld_t=Ld_t, Lq_t=Lq_t, Td0_t=Td0_t, Tq0_t=Tq0_t)		
    elif (gen_model=="6aOrder"):
        gen = comps_module.SynchronGenerator6aOrderVBR('gen', dpsimpy.LogLevel.off)
        gen.set_operational_parameters_per_unit(nom_power=nom_power, nom_voltage=nominal_voltage_mv, nom_frequency=frequency, 
                                                H=H, Ld=Ld, Lq=Lq, L0=L0, Ld_t=Ld_t, Lq_t=Lq_t, Td0_t=Td0_t, Tq0_t=Tq0_t,
                                                Ld_s=Ld_s, Lq_s=Lq_s, Td0_s=Td0_s, Tq0_s=Tq0_s, Taa=Taa)	
    elif (gen_model=="6bOrder"):
        gen = comps_module.SynchronGenerator6bOrderVBR('gen', dpsimpy.LogLevel.off)
        gen.set_operational_parameters_per_unit(nom_power=nom_power, nom_voltage=nominal_voltage_mv, nom_frequency=frequency, 
                                                H=H, Ld=Ld, Lq=Lq, L0=L0, Ld_t=Ld_t, Lq_t=Lq_t, Td0_t=Td0_t, Tq0_t=Tq0_t,
                                                Ld_s=Ld_s, Lq_s=Lq_s, Td0_s=Td0_s, Tq0_s=Tq0_s)		
    gen.set_initial_values(init_complex_electrical_power=initial_electrical_power, init_mechanical_power=initial_electrical_power.real, 
                           init_complex_terminal_voltage=initial_terminal_voltage)

    # exciter
    if with_exciter:
        exciter = dpsimpy.signal.ExciterDC1Simp("Exciter", dpsimpy.LogLevel.off)
        exciter.set_parameters(exciter_parameters)
        gen.add_exciter(exciter)
        
        # PSS
        if with_pss:
            pss = dpsimpy.signal.PSS1A("PSS", dpsimpy.LogLevel.info)
            pss.set_parameters(pss_parameters)
            gen.add_pss(pss)
            
    # governor
    if with_governor:
        governor = dpsimpy.signal.TurbineGovernorType1("Governor", dpsimpy.LogLevel.off)
        governor.set_parameters(governor_parameters)
        gen.add_governor(governor)
    
    # Load
    load = None
    if domain == dpsimpy.Domain.SP:
        load = comps_module.Load("Load", dpsimpy.LogLevel.off)
        load.set_parameters(active_power, reactive_power, nominal_voltage_mv)
    elif domain == dpsimpy.Domain.DP:
        load = comps_module.RXLoad("Load", dpsimpy.LogLevel.off)
        load.set_parameters(active_power, reactive_power, nominal_voltage_mv)
    elif domain == dpsimpy.Domain.EMT:
        load = comps_module.RXLoad("Load", dpsimpy.LogLevel.off)
        load.set_parameters(dpsimpy.Math.single_phase_parameter_to_three_phase(active_power/3),
                            dpsimpy.Math.single_phase_parameter_to_three_phase(reactive_power/3),
                            nominal_voltage_mv)
 
    # Load Step as Switch
    switch = None
    gnd = None
    if (domain == dpsimpy.Domain.SP or domain == dpsimpy.Domain.DP):
        switch = comps_module.Switch('Switch', dpsimpy.LogLevel.off)
        gnd = dpsimpy.dp.SimNode.gnd
    elif domain == dpsimpy.Domain.EMT:
        switch = comps_module.SeriesSwitch('Switch', dpsimpy.LogLevel.off)
        gnd = dpsimpy.emt.SimNode.gnd 
    switch.set_parameters(switch_open, switch_closed)
    switch.open()
    
    ### Connections
    gen.connect([n1])
    load.connect([n1])
    switch.connect([gnd, n1])
    
    ### Define system topology
    system = dpsimpy.SystemTopology(frequency, [n1], [gen, load, switch])

    ### Logging
    logger = dpsimpy.Logger(name)
    logger.log_attribute('v_gen', 'v_intf', gen)
    logger.log_attribute('i_gen', 'i_intf', gen)
    logger.log_attribute('Te', 'Te', gen)
    logger.log_attribute('Tm', 'Tm', gen)
    logger.log_attribute('Ef', 'Ef', gen)
    
    ### Simulation
    sim = dpsimpy.Simulation(name, dpsimpy.LogLevel.off)
    sim.set_system(system)
    sim.do_init_from_nodes_and_terminals(True)
    sim.set_domain(domain)
    sim.set_direct_solver_implementation(dpsimpy.DirectLinearSolverImpl.SparseLU)
    sim.set_time_step(time_step)
    sim.set_final_time(final_time)
    sim.do_system_matrix_recomputation(True)
    
    sw_event_1 = dpsimpy.event.SwitchEvent(1.0, switch, True)
    sim.add_event(sw_event_1)
    
    sw_event_2 = dpsimpy.event.SwitchEvent(4, switch, False)
    sim.add_event(sw_event_2)

    sim.add_logger(logger)
    sim.run()
    
    return name

## Declare plot function

In [ ]:
width = 16
height = 4
color_EMT = 'black'
color_SP = 'C10'
color_DP = 'C11'
linestyle_EMT = '-'
linestyle_SP = '-'
linestyle_DP = 'dashed'
common_time=100e-6
roi_start=0
roi_end=4.5
roi = range(int(roi_start/common_time), int(roi_end/common_time))

def plot_mechVar(varname, ylabel, ts_dpsim_EMT, ts_dpsim_SP, ts_dpsim_DP):
        
    time = ts_dpsim_EMT[varname].interpolate(common_time).time[roi]
    values_emt = ts_dpsim_EMT[varname].interpolate(common_time).values[roi]
    values_sp = ts_dpsim_SP[varname].interpolate(common_time).values[roi]
    values_dp = ts_dpsim_DP[varname].interpolate(common_time).values[roi]
        
    fig = plt.figure(figsize=(width, height))
    plt.plot(time, values_emt, label='EMT', linestyle=linestyle_EMT, color=color_EMT, linewidth=3)    
    plt.plot(time, values_sp, label='SP',  color=color_SP, linestyle=linestyle_SP,  linewidth=3) 
    plt.plot(time, values_dp, label='DP',  color=color_DP, linestyle=linestyle_DP, linewidth=3)     
    plt.ylabel(ylabel)
    plt.xlabel('Time (s)')
    plt.xlim(0, 4.5)
    plt.grid()
    plt.legend(loc='lower right')
    plt.show()
    
    #calculate RMSE
    nom_value = 0.540541 # power flow result
    rmse_emt = ts_dpsim_SP[varname].rmse(ts_dpsim_SP[varname].interpolate(common_time), ts_dpsim_EMT[varname].interpolate(common_time))
    print('RMSE EMT = {:.6f} (pu), which is {:.3f}% of the nominal value = {:.3f} (pu) '.format(rmse_emt, rmse_emt/nom_value*100, nom_value))
    rmse_dp = ts_dpsim_SP[varname].rmse(ts_dpsim_SP[varname].interpolate(common_time), ts_dpsim_DP[varname].interpolate(common_time))
    print('RMSE DP = {:.6f} (pu), which is {:.3f}% of the nominal value = {:.3f} (pu) '.format(rmse_dp, rmse_dp/nom_value*100, nom_value))
    
    return (rmse_emt/nom_value*100, rmse_dp/nom_value*100)
    
    
def plot_elecVar(variable_EMT, variable_SP, variable_DP, ylabel, ts_dpsim_EMT, ts_dpsim_SP_shift, ts_dpsim_DP_shift):    
    time = ts_dpsim_EMT[variable_EMT].interpolate(common_time).time[roi]
    values_emt = ts_dpsim_EMT[variable_EMT].interpolate(common_time).values[roi]
    values_sp = np.sqrt(2/3) * ts_dpsim_SP_shift[variable_SP].interpolate(common_time).values[roi]
    values_dp = np.sqrt(2/3) * ts_dpsim_DP_shift[variable_DP].interpolate(common_time).values[roi] 
        
    fig = plt.figure(figsize=(width, height))
    plt.plot(time, values_emt, label='EMT', linestyle=linestyle_EMT, color=color_EMT, linewidth=3)    
    plt.plot(time, values_sp, label='SP',  color=color_SP, linestyle=linestyle_SP,  linewidth=3) 
    plt.plot(time, values_dp, label='DP',  color=color_DP, linestyle=linestyle_DP, linewidth=3) 
    plt.ylabel(ylabel)
    plt.xlabel('Time (s)')
    plt.xlim(0.9, 1.2)
    plt.grid()
    plt.legend(loc='lower right')
    plt.show()
    
    #calculate RMSE
    ts_dpsim_SP_real = TimeSeries(variable_SP, ts_dpsim_SP_shift[variable_SP].time, np.sqrt(2/3) * ts_dpsim_SP_shift[variable_SP].values.real)
    rmse_emt = ts_dpsim_SP_real.rmse(ts_dpsim_SP_real.interpolate(common_time), ts_dpsim_EMT[variable_EMT].interpolate(common_time))
    nom_value = abs(max(ts_dpsim_SP_shift[variable_SP].values[0:int(0.5/common_time)]))
    print('RMSE EMT = {:.6f} (pu), which is {:.3f}% of the nominal value = {:.3f} (pu) '.format(rmse_emt, rmse_emt/nom_value*100, nom_value))
    
    ts_dpsim_DP_real = TimeSeries(variable_DP, ts_dpsim_DP_shift[variable_DP].time, np.sqrt(2/3) * ts_dpsim_DP_shift[variable_DP].values.real)
    rmse_dp = ts_dpsim_SP_real.rmse(ts_dpsim_SP_real.interpolate(common_time), ts_dpsim_DP_real.interpolate(common_time))
    print('RMSE DP = {:.6f} (pu), which is {:.3f}% of the nominal value = {:.3f} (pu) '.format(rmse_dp, rmse_dp/nom_value*100, nom_value))
    
    return (rmse_emt/nom_value*100, rmse_dp/nom_value*100)

def plot_elecVar2(variable_EMT, variable_SP, variable_DP, ylabel, ts_dpsim_EMT, ts_dpsim_SP, ts_dpsim_DP):   
    time = ts_dpsim_EMT[variable_EMT].interpolate(common_time).time[roi]
    values_emt = ts_dpsim_EMT[variable_EMT].interpolate(common_time).values[roi]
    values_sp = ts_dpsim_SP[variable_SP].interpolate(common_time).values[roi]
    values_dp = ts_dpsim_DP[variable_DP].interpolate(common_time).values[roi] 
        
    fig = plt.figure(figsize=(width, height))
    plt.plot(time, values_emt, label='EMT', linestyle=linestyle_EMT, color=color_EMT, linewidth=3)    
    plt.plot(time, values_sp, label='SP',  color=color_SP, linestyle=linestyle_SP,  linewidth=3) 
    plt.plot(time, values_dp, label='DP',  color=color_DP, linestyle=linestyle_DP, linewidth=3) 
    plt.ylabel(ylabel)
    plt.xlabel('Time (s)')
    plt.xlim(0.9, 4.5)
    plt.grid()
    plt.legend(loc='lower right')
    plt.show()
    
    #calculate RMSE
    ts_dpsim_SP_real = TimeSeries(variable_SP, ts_dpsim_SP[variable_SP].time, ts_dpsim_SP[variable_SP].values.real)
    rmse_emt = ts_dpsim_SP_real.rmse(ts_dpsim_SP_real.interpolate(common_time), ts_dpsim_EMT[variable_EMT].interpolate(common_time))
    nom_value = abs(max(ts_dpsim_SP[variable_SP].values[0:int(0.5/common_time)]))
    print('RMSE EMT = {:.6f} (pu), which is {:.3f}% of the nominal value = {:.3f} (pu) '.format(rmse_emt, rmse_emt/nom_value*100, nom_value))
    
    ts_dpsim_DP_real = TimeSeries(variable_DP, ts_dpsim_DP[variable_DP].time, ts_dpsim_DP[variable_DP].values.real)
    rmse_dp = ts_dpsim_SP_real.rmse(ts_dpsim_SP_real.interpolate(common_time), ts_dpsim_DP_real.interpolate(common_time))
    print('RMSE DP = {:.6f} (pu), which is {:.3f}% of the nominal value = {:.3f} (pu) '.format(rmse_dp, rmse_dp/nom_value*100, nom_value))
    
    return (rmse_emt/nom_value*100, rmse_dp/nom_value*100)

## 1. Run DPsim Simulations with Exciter and Load Results

In [ ]:
log_name_sp = reducedOrderSG_LoadStep(dpsimpy.Domain.SP, gen_model="4Order", with_exciter=True, with_pss=False, with_governor=False, final_time=5, time_step=0.001)
log_name_dp = reducedOrderSG_LoadStep(dpsimpy.Domain.DP, gen_model="4Order", with_exciter=True, with_pss=False, with_governor=False, final_time=5, time_step=0.001)
log_name_emt = reducedOrderSG_LoadStep(dpsimpy.Domain.EMT, gen_model="4Order", with_exciter=True, with_pss=False, with_governor=False, final_time=5, time_step=0.001)

work_dir = os.getcwd() + "/logs/"

ts_dpsim_SP = None
ts_dpsim_SP_shift = None
ts_dpsim_DP = None
ts_dpsim_DP_shift = None
ts_dpsim_EMT = None

for i in range(3):
    path_logfile = work_dir + log_name_sp + "/" + log_name_sp + ".csv"
    ts_dpsim_SP = read_timeseries_dpsim(path_logfile)
    ts_dpsim_SP_shift = ts.frequency_shift_list(ts_dpsim_SP, 60)

    path_logfile = work_dir + log_name_dp + "/" + log_name_dp + ".csv"
    ts_dpsim_DP = read_timeseries_dpsim(path_logfile)
    ts_dpsim_DP_shift = ts.frequency_shift_list(ts_dpsim_DP, 60)
    
    path_logfile = work_dir + log_name_emt + "/" + log_name_emt + ".csv"
    ts_dpsim_EMT = read_timeseries_dpsim(path_logfile)

### Plot Results

#### Electrical torque

#### 3th order SG

In [ ]:
rmse_emt_te, rmse_dp_te = plot_mechVar("Te", "Electrical torque (p.u)", ts_dpsim_EMT, ts_dpsim_SP, ts_dpsim_DP)

### Field Voltage

In [ ]:
rmse_emt_vf, rmse_dp_vf = plot_mechVar("Ef", "Field Voltage (p.u)", ts_dpsim_EMT, ts_dpsim_SP, ts_dpsim_DP)

### Generator Current

In [ ]:
rmse_emt_current, rmse_dp_current = plot_elecVar('i_gen_0', 'i_gen_shift', 'i_gen_shift', 'Current (A)', ts_dpsim_EMT, ts_dpsim_SP_shift, ts_dpsim_DP_shift)

### Generator Voltage

In [ ]:
rmse_emt_voltage, rmse_dp_voltage = plot_elecVar('v_gen_0', 'v_gen_shift', 'v_gen_shift', 'Voltage (V)', ts_dpsim_EMT, ts_dpsim_SP_shift, ts_dpsim_DP_shift)

### Assertion

In [ ]:
tol = 0.001 # %

assert(rmse_emt_current<tol)
assert(rmse_emt_voltage<tol)
assert(rmse_emt_vf<tol)
assert(rmse_emt_te<tol)
assert(rmse_dp_current<tol)
assert(rmse_dp_voltage<tol)
assert(rmse_dp_te<tol)
assert(rmse_dp_vf<tol)

## 2. Run DPsim Simulations with Exciter and PSS and Load Results

In [ ]:
log_name_sp = reducedOrderSG_LoadStep(dpsimpy.Domain.SP, gen_model="4Order", with_exciter=True, with_pss=True, with_governor=False, final_time=5, time_step=0.001)
log_name_dp = reducedOrderSG_LoadStep(dpsimpy.Domain.DP, gen_model="4Order", with_exciter=True, with_pss=True, with_governor=False, final_time=5, time_step=0.001)
log_name_emt = reducedOrderSG_LoadStep(dpsimpy.Domain.EMT, gen_model="4Order", with_exciter=True, with_pss=True, with_governor=False, final_time=5, time_step=0.001)

work_dir = os.getcwd() + "/logs/"

ts_dpsim_SP = None
ts_dpsim_SP_shift = None
ts_dpsim_DP = None
ts_dpsim_DP_shift = None
ts_dpsim_EMT = None

for i in range(3):
    path_logfile = work_dir + log_name_sp + "/" + log_name_sp + ".csv"
    ts_dpsim_SP = read_timeseries_dpsim(path_logfile)
    ts_dpsim_SP_shift = ts.frequency_shift_list(ts_dpsim_SP, 60)

    path_logfile = work_dir + log_name_dp + "/" + log_name_dp + ".csv"
    ts_dpsim_DP = read_timeseries_dpsim(path_logfile)
    ts_dpsim_DP_shift = ts.frequency_shift_list(ts_dpsim_DP, 60)
    
    path_logfile = work_dir + log_name_emt + "/" + log_name_emt + ".csv"
    ts_dpsim_EMT = read_timeseries_dpsim(path_logfile)

### Plot Results

#### Electrical Torque

In [ ]:
rmse_emt_te, rmse_dp_te = plot_mechVar("Te", "Electrical torque (p.u)", ts_dpsim_EMT, ts_dpsim_SP, ts_dpsim_DP)

### Field Voltage

In [ ]:
rmse_emt_vf, rmse_dp_vf = plot_mechVar("Ef", "Field Voltage (p.u)", ts_dpsim_EMT, ts_dpsim_SP, ts_dpsim_DP)

### Current

In [ ]:
rmse_emt_current, rmse_dp_current = plot_elecVar('i_gen_0', 'i_gen_shift', 'i_gen_shift', 'Current (A)', ts_dpsim_EMT, ts_dpsim_SP_shift, ts_dpsim_DP_shift)

### Voltage

In [ ]:
rmse_emt_voltage, rmse_dp_voltage = plot_elecVar('v_gen_0', 'v_gen_shift', 'v_gen_shift', 'Voltage (V)', ts_dpsim_EMT, ts_dpsim_SP_shift, ts_dpsim_DP_shift)

### Assertion

In [ ]:
tol = 0.001 # %

assert(rmse_emt_current<tol)
assert(rmse_emt_voltage<tol)
assert(rmse_emt_vf<tol)
assert(rmse_emt_te<tol)
assert(rmse_dp_current<tol)
assert(rmse_dp_voltage<tol)
assert(rmse_dp_te<tol)
assert(rmse_dp_vf<tol)

## 3. Run DPsim Simulations with Exciter, PSS and Turbine-Governor and Load Results

In [ ]:
log_name_sp = reducedOrderSG_LoadStep(dpsimpy.Domain.SP, gen_model="4Order", with_exciter=True, with_pss=True, with_governor=True, final_time=5, time_step=0.001)
log_name_dp = reducedOrderSG_LoadStep(dpsimpy.Domain.DP, gen_model="4Order", with_exciter=True, with_pss=True, with_governor=True, final_time=5, time_step=0.001)
log_name_emt = reducedOrderSG_LoadStep(dpsimpy.Domain.EMT, gen_model="4Order", with_exciter=True, with_pss=True, with_governor=True, final_time=5, time_step=0.001)

work_dir = os.getcwd() + "/logs/"

ts_dpsim_SP = None
ts_dpsim_SP_shift = None
ts_dpsim_DP = None
ts_dpsim_DP_shift = None
ts_dpsim_EMT = None

for i in range(3):
    path_logfile = work_dir + log_name_sp + "/" + log_name_sp + ".csv"
    ts_dpsim_SP = read_timeseries_dpsim(path_logfile)
    ts_dpsim_SP_shift = ts.frequency_shift_list(ts_dpsim_SP, 60)

    path_logfile = work_dir + log_name_dp + "/" + log_name_dp + ".csv"
    ts_dpsim_DP = read_timeseries_dpsim(path_logfile)
    ts_dpsim_DP_shift = ts.frequency_shift_list(ts_dpsim_DP, 60)
    
    path_logfile = work_dir + log_name_emt + "/" + log_name_emt + ".csv"
    ts_dpsim_EMT = read_timeseries_dpsim(path_logfile)

### Plot Results

#### Electrical Torque

In [ ]:
rmse_emt_te, rmse_dp_te = plot_mechVar("Te", "Electrical torque (p.u)", ts_dpsim_EMT, ts_dpsim_SP, ts_dpsim_DP)

#### Mechanical Torque

In [ ]:
rmse_emt_tm, rmse_dp_tm = plot_mechVar("Tm", "Mechanical torque (p.u)", ts_dpsim_EMT, ts_dpsim_SP, ts_dpsim_DP)

#### Field Voltage

In [ ]:
rmse_emt_vf, rmse_dp_vf = plot_mechVar("Ef", "Field Voltage (p.u)", ts_dpsim_EMT, ts_dpsim_SP, ts_dpsim_DP)

#### Current

In [ ]:
rmse_emt_current, rmse_dp_current = plot_elecVar('i_gen_0', 'i_gen_shift', 'i_gen_shift', 'Current (A)', ts_dpsim_EMT, ts_dpsim_SP_shift, ts_dpsim_DP_shift)

#### Voltage

In [ ]:
rmse_emt_voltage, rmse_dp_voltage = plot_elecVar('v_gen_0', 'v_gen_shift', 'v_gen_shift', 'Voltage (V)', ts_dpsim_EMT, ts_dpsim_SP_shift, ts_dpsim_DP_shift)

### Assert

In [ ]:
tol = 0.001 # %

assert(rmse_emt_current<tol)
assert(rmse_emt_voltage<tol)
assert(rmse_emt_vf<tol)
assert(rmse_emt_te<tol)
assert(rmse_emt_tm<tol)
assert(rmse_dp_current<tol)
assert(rmse_dp_voltage<tol)
assert(rmse_dp_te<tol)
assert(rmse_dp_tm<tol)
assert(rmse_dp_vf<tol)